<a href="https://colab.research.google.com/github/syakesaba/jupyter-notebooks/blob/main/iceberg_duckdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Secretの読み込み

In [123]:
from google.colab import userdata

S3A_ENDPOINT = userdata.get('S3A_ENDPOINT') # xxxxxxx.compat.objectstorage.{S3A_REGION}.oraclecloud.com
S3A_ACCESS_KEY = userdata.get('S3A_ACCESS_KEY') # xxxxxxxxxxxx
S3A_SECRET_KEY = userdata.get('S3A_SECRET_KEY') # xxxxxxxxxxxx
S3A_REGION = userdata.get('S3A_REGION') # ap-northeast-1

# NSRLJPデータセットの取得

In [ ]:
!wget https://www.kazamiya.net/files/projects/NSRLJP_202408.7z
!7z x NSRLJP_202408.7z
!mv ./NSRLJP_202408/NSRLFile{.txt,.csv}
!mv ./NSRLJP_202408/NSRLMfg{.txt,.csv}
!mv ./NSRLJP_202408/NSRLOS{.txt,.csv}
!mv ./NSRLJP_202408/NSRLProd{.txt,.csv}

--2025-03-16 13:19:08--  https://www.kazamiya.net/files/projects/NSRLJP_202408.7z
Resolving www.kazamiya.net (www.kazamiya.net)... 203.143.110.17
Connecting to www.kazamiya.net (www.kazamiya.net)|203.143.110.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319470599 (305M) [application/x-7z-compressed]
Saving to: ‘NSRLJP_202408.7z.1’

NSRLJP_202408.7z.1  100%[===================>] 304.67M  11.1MB/s    in 33s     

2025-03-16 13:19:42 (9.15 MB/s) - ‘NSRLJP_202408.7z.1’ saved [319470599/319470599]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 319470599 bytes (305 MiB)

Extracting archive: NSRLJP_202408.7z
--
Path = NSRLJP_202408.7z
Type = 7z
Physical Size = 319470599
Headers Size = 275
Method = LZMA2:24
Solid = +
Blocks = 1

  0% 

## データセットをduckdbにimport

In [ ]:
!pip install -qq duckdb

In [ ]:
#!/usr/bin/env python
# encoding: utf-8

import duckdb
db = duckdb.connect("duck.db")
db.sql("""
-- 1. メーカーマスタテーブルの作成
DROP TABLE IF EXISTS NSRLMfg;
CREATE TABLE IF NOT EXISTS NSRLMfg (
    MfgCode TEXT PRIMARY KEY,
    MfgName TEXT NOT NULL
);

-- 2. OSマスタテーブルの作成
DROP TABLE IF EXISTS NSRLOS;
CREATE TABLE IF NOT EXISTS NSRLOS (
    OpSystemCode TEXT PRIMARY KEY,
    OpSystemName TEXT NOT NULL,
    OpSystemVersion TEXT,
    MfgCode TEXT,
);

-- 3. 製品マスタテーブルの作成
DROP TABLE IF EXISTS NSRLProd;
CREATE TABLE IF NOT EXISTS NSRLProd (
    ProductCode TEXT PRIMARY KEY,
    ProductName TEXT NOT NULL,
    ProductVersion TEXT,
    OpSystemCode TEXT,
    MfgCode TEXT,
    Language TEXT,
    ApplicationType TEXT,
);

-- 4. NSRLファイル情報テーブルの作成
DROP TABLE IF EXISTS NSRLFile;
CREATE TABLE IF NOT EXISTS NSRLFile (
    SHA1 TEXT PRIMARY KEY,
    MD5 TEXT NOT NULL,
    CRC32 TEXT NOT NULL,
    FileName TEXT NOT NULL,
    FileSize TEXT NOT NULL,
    ProductCode TEXT,
    OpSystemCode TEXT,
    SpecialCode TEXT,
);

-- 5. インデックスの作成
CREATE INDEX IF NOT EXISTS idx_md5 ON NSRLFile (MD5);

-- 6. データのインポート
COPY NSRLMfg FROM './NSRLJP_202408/NSRLMfg.csv';
COPY NSRLProd FROM './NSRLJP_202408/NSRLProd.csv';
COPY NSRLOS FROM './NSRLJP_202408/NSRLOS.csv';
COPY NSRLFile FROM './NSRLJP_202408/NSRLFile.csv';
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## データが入っているか確認

In [ ]:
db.sql("""
SELECT
    *
FROM
    NSRLFile
LEFT JOIN
    NSRLProd ON NSRLFile.ProductCode = NSRLProd.ProductCode
LEFT JOIN
    NSRLOS ON NSRLFile.OpSystemCode = NSRLOS.OpSystemCode
LEFT JOIN
    NSRLMfg ON NSRLOS.MfgCode = NSRLMfg.MfgCode
WHERE
    SHA1 = '003f1b9a60927b2c429412cb4698907e0a0c68ce'
AND
    NSRLFile.OpSystemCode = '1010'
""")

┌──────────────────────────────────────────┬──────────────────────────────────┬──────────┬─────────────┬──────────┬─────────────┬──────────────┬─────────────┬─────────────┬──────────────────┬────────────────┬──────────────┬─────────┬────────────┬────────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬─────────┬───────────┐
│                   SHA1                   │               MD5                │  CRC32   │  FileName   │ FileSize │ ProductCode │ OpSystemCode │ SpecialCode │ ProductCode │   ProductName    │ ProductVersion │ OpSystemCode │ MfgCode │  Language  │  ApplicationType   │ OpSystemCode │ OpSystemName │ OpSystemVersion │ MfgCode │ MfgCode │  MfgName  │
│                 varchar                  │             varchar              │ varchar  │   varchar   │ varchar  │   varchar   │   varchar    │   varchar   │   varchar   │     varchar      │    varchar     │   varchar    │ varchar │  varchar   │      varchar       │   varchar    │   varchar    │     va

# ApacheSparkの利用

## OpenJDK 17 (Java7)のインストール

In [ ]:
# Java https://jdk.java.net/archive/
!wget https://download.java.net/java/GA/jdk17/0d483333a00540d886896bac774ff48b/35/GPL/openjdk-17_linux-x64_bin.tar.gz
!tar xzf openjdk-17_linux-x64_bin.tar.gz
%env JAVA_HOME=/content/jdk-17

--2025-03-16 13:20:35--  https://download.java.net/java/GA/jdk17/0d483333a00540d886896bac774ff48b/35/GPL/openjdk-17_linux-x64_bin.tar.gz
Resolving download.java.net (download.java.net)... 23.2.72.114
Connecting to download.java.net (download.java.net)|23.2.72.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186661523 (178M) [application/x-gzip]
Saving to: ‘openjdk-17_linux-x64_bin.tar.gz.4’

openjdk-17_linux-x6 100%[===================>] 178.01M   113MB/s    in 1.6s    

2025-03-16 13:20:36 (113 MB/s) - ‘openjdk-17_linux-x64_bin.tar.gz.4’ saved [186661523/186661523]

env: JAVA_HOME=/content/jdk-17


## PySparkのインストール

In [ ]:
!pip install -qq pyspark==3.5.5

## SparkSessionの作成、icebergテーブルの作成とデータのアップロード

In [ ]:
#!/usr/bin/env python
# encoding: utf-8

import os
from pyspark.sql import SparkSession

CATALOG_NAME = "catalog_iceberg"
WAREHOUSE_URL = "s3a://datalakehouse/iceberg"

spark: SparkSession = (
    SparkSession.builder.master("local[*,3]")
    .appName("app_nsrljp")
    # .config("spark.driver.memory", "2g")
    # .config("spark.executor.memory", "2g")
    # .config("spark.log.level", "DEBUG")
    .config(
        "spark.jars.packages",
        f"org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1"
        f",org.apache.hadoop:hadoop-aws:3.3.4",
    )
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .config(
        f"spark.sql.catalog.{CATALOG_NAME}", "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(
        f"spark.sql.catalog.{CATALOG_NAME}.type", "hadoop"
    )  # https://iceberg.apache.org/docs/1.5.0/spark-configuration/
    .config(f"spark.sql.catalog.{CATALOG_NAME}.warehouse", f"{WAREHOUSE_URL}")
    .config(f"spark.sql.defaultCatalog", f"{CATALOG_NAME}")
    .config("spark.hadoop.fs.s3a.endpoint.region", S3A_REGION or "")
    .config("spark.hadoop.fs.s3a.endpoint", S3A_ENDPOINT or "")
    .config("spark.hadoop.fs.s3a.access.key", S3A_ACCESS_KEY or "")
    .config("spark.hadoop.fs.s3a.secret.key", S3A_SECRET_KEY or "")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

DATABASE_NAME = "NSRLJP"

spark.sql(f"""DROP TABLE IF EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg PURGE""")

spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg (
    MfgCode STRING,
    MfgName STRING
) USING ICEBERG
LOCATION '{WAREHOUSE_URL}/{DATABASE_NAME}/NSRLMfg/'
"""
)

spark.sql(f"""DROP TABLE IF EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS PURGE""")

spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS (
    OpSystemCode STRING,
    OpSystemName STRING,
    OpSystemVersion STRING,
    MfgCode STRING
) USING ICEBERG
LOCATION '{WAREHOUSE_URL}/{DATABASE_NAME}/NSRLOS/'
"""
)

spark.sql(f"""DROP TABLE IF EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd PURGE""")

spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd (
    ProductCode STRING,
    ProductName STRING,
    ProductVersion STRING,
    OpSystemCode STRING,
    MfgCode STRING,
    Language STRING,
    ApplicationType STRING
) USING ICEBERG
LOCATION '{WAREHOUSE_URL}/{DATABASE_NAME}/NSRLProd/'
"""
)

spark.sql(f"""DROP TABLE IF EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile PURGE""")

spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile (
    `SHA-1` STRING,
    MD5 STRING,
    CRC32 STRING,
    FileName STRING,
    FileSize STRING,
    ProductCode STRING,
    OpSystemCode STRING,
    SpecialCode STRING
) USING ICEBERG
PARTITIONED BY (OpSystemCode)
LOCATION '{WAREHOUSE_URL}/{DATABASE_NAME}/NSRLFile/'
"""
)

spark.read.option("inferSchema", True).option("header", True).csv(
    "./NSRLJP_202408/NSRLMfg.csv"
).writeTo(f"{CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg").overwritePartitions()

spark.read.option("inferSchema", True).option("header", True).csv(
    "./NSRLJP_202408/NSRLOS.csv"
).writeTo(f"{CATALOG_NAME}.{DATABASE_NAME}.NSRLOS").overwritePartitions()

spark.read.option("inferSchema", True).option("header", True).csv(
    "./NSRLJP_202408/NSRLProd.csv"
).writeTo(f"{CATALOG_NAME}.{DATABASE_NAME}.NSRLProd").overwritePartitions()

spark.read.option("inferSchema", True).option("header", True).csv(
    "./NSRLJP_202408/NSRLFile.csv"
).writeTo(f"{CATALOG_NAME}.{DATABASE_NAME}.NSRLFile").overwritePartitions()

`You are using approximately 326.79 MiB of the 20 GiB limit of free combined Object Storage and Archive Storage. Upgrade to use unlimited storage.`

## PySparkによるクエリ

In [ ]:
# 通常のクエリ
spark.sql(
    f"""
SELECT
    *
FROM
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.ProductCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd.ProductCode
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.OpSystemCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS.OpSystemCode
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS.MfgCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg.MfgCode
WHERE
    `SHA-1` = '003f1b9a60927b2c429412cb4698907e0a0c68ce'
"""
).show()

+--------------------+--------------------+--------+-----------+--------+-----------+------------+-----------+-----------+--------------+--------------+------------+-------+--------+----------------+------------+------------+---------------+-------+-------+---------+
|               SHA-1|                 MD5|   CRC32|   FileName|FileSize|ProductCode|OpSystemCode|SpecialCode|ProductCode|   ProductName|ProductVersion|OpSystemCode|MfgCode|Language| ApplicationType|OpSystemCode|OpSystemName|OpSystemVersion|MfgCode|MfgCode|  MfgName|
+--------------------+--------------------+--------+-----------+--------+-----------+------------+-----------+-----------+--------------+--------------+------------+-------+--------+----------------+------------+------------+---------------+-------+-------+---------+
|003f1b9a60927b2c4...|220a7215a63faa737...|5E9769BF|Desktop.ini|     798|      50047|        1010|       NULL|      50047|Windows 10 x86|         2015-|        1010|   5001|Japanese|Operating Syst

In [ ]:
# パーティション（フォルダ）による最適化が走るクエリ
spark.sql(
    f"""
SELECT
    *
FROM
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.ProductCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLProd.ProductCode
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.OpSystemCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS.OpSystemCode
LEFT JOIN
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg ON {CATALOG_NAME}.{DATABASE_NAME}.NSRLOS.MfgCode = {CATALOG_NAME}.{DATABASE_NAME}.NSRLMfg.MfgCode
WHERE
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.`SHA-1` = '003f1b9a60927b2c429412cb4698907e0a0c68ce'
AND
    {CATALOG_NAME}.{DATABASE_NAME}.NSRLFile.OpSystemCode = '1010'
"""
).show()

+--------------------+--------------------+--------+-----------+--------+-----------+------------+-----------+-----------+--------------+--------------+------------+-------+--------+----------------+------------+------------+---------------+-------+-------+---------+
|               SHA-1|                 MD5|   CRC32|   FileName|FileSize|ProductCode|OpSystemCode|SpecialCode|ProductCode|   ProductName|ProductVersion|OpSystemCode|MfgCode|Language| ApplicationType|OpSystemCode|OpSystemName|OpSystemVersion|MfgCode|MfgCode|  MfgName|
+--------------------+--------------------+--------+-----------+--------+-----------+------------+-----------+-----------+--------------+--------------+------------+-------+--------+----------------+------------+------------+---------------+-------+-------+---------+
|003f1b9a60927b2c4...|220a7215a63faa737...|5E9769BF|Desktop.ini|     798|      50047|        1010|       NULL|      50047|Windows 10 x86|         2015-|        1010|   5001|Japanese|Operating Syst

## DuckDBによるクエリ

In [ ]:
# Extensionsをロード
import duckdb
duckdb.sql("""
INSTALL httpfs;
INSTALL iceberg;
LOAD iceberg;
LOAD httpfs;
""")

# S3用クレデンシャルを作成

duckdb.sql(f"""
DROP SECRET IF EXISTS oci_storage;
CREATE SECRET IF NOT EXISTS  oci_storage (
    TYPE s3,
    REGION '{S3A_REGION}',
    ENDPOINT '{S3A_ENDPOINT}',
    KEY_ID '{S3A_ACCESS_KEY}',
    SECRET '{S3A_SECRET_KEY}',
    URL_STYLE 'path'
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [ ]:
# 通常のクエリ
duckdb.sql("""
WITH NSRLFile AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLFile',
            allow_moved_paths = true
        )
), NSRLProd AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLProd',
            allow_moved_paths = true
        )
), NSRLOS AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLOS',
            allow_moved_paths = true
        )
), NSRLMfg AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLMfg',
            allow_moved_paths = true
        )
)
SELECT
    *
FROM
    NSRLFile
LEFT JOIN
    NSRLProd ON NSRLFile.ProductCode = NSRLProd.ProductCode
LEFT JOIN
    NSRLOS ON NSRLFile.OpSystemCode = NSRLOS.OpSystemCode
LEFT JOIN
    NSRLMfg ON NSRLOS.MfgCode = NSRLMfg.MfgCode
WHERE
    NSRLFile."SHA-1" = '003f1b9a60927b2c429412cb4698907e0a0c68ce'
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────┬──────────────────────────────────┬──────────┬─────────────┬──────────┬─────────────┬──────────────┬─────────────┬─────────────┬────────────────┬────────────────┬──────────────┬─────────┬──────────┬──────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬─────────┬───────────┐
│                  SHA-1                   │               MD5                │  CRC32   │  FileName   │ FileSize │ ProductCode │ OpSystemCode │ SpecialCode │ ProductCode │  ProductName   │ ProductVersion │ OpSystemCode │ MfgCode │ Language │ ApplicationType  │ OpSystemCode │ OpSystemName │ OpSystemVersion │ MfgCode │ MfgCode │  MfgName  │
│                 varchar                  │             varchar              │ varchar  │   varchar   │ varchar  │   varchar   │   varchar    │   varchar   │   varchar   │    varchar     │    varchar     │   varchar    │ varchar │ varchar  │     varchar      │   varchar    │   varchar    │     varchar     │ varcha

In [ ]:
# パーティション（フォルダ）による最適化が走るクエリ
duckdb.sql("""
WITH NSRLFile AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLFile',
            allow_moved_paths = true
        )
), NSRLProd AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLProd',
            allow_moved_paths = true
        )
), NSRLOS AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLOS',
            allow_moved_paths = true
        )
), NSRLMfg AS (
    SELECT
        *
    FROM
        iceberg_scan(
            's3://datalakehouse/iceberg/NSRLJP/NSRLMfg',
            allow_moved_paths = true
        )
)
SELECT
    *
FROM
    NSRLFile
LEFT JOIN
    NSRLProd ON NSRLFile.ProductCode = NSRLProd.ProductCode
LEFT JOIN
    NSRLOS ON NSRLFile.OpSystemCode = NSRLOS.OpSystemCode
LEFT JOIN
    NSRLMfg ON NSRLOS.MfgCode = NSRLMfg.MfgCode
WHERE
    NSRLFile."SHA-1" = '003f1b9a60927b2c429412cb4698907e0a0c68ce'
AND
    NSRLFile.OpSystemCode = '1010'
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────┬──────────────────────────────────┬──────────┬─────────────┬──────────┬─────────────┬──────────────┬─────────────┬─────────────┬────────────────┬────────────────┬──────────────┬─────────┬──────────┬──────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬─────────┬───────────┐
│                  SHA-1                   │               MD5                │  CRC32   │  FileName   │ FileSize │ ProductCode │ OpSystemCode │ SpecialCode │ ProductCode │  ProductName   │ ProductVersion │ OpSystemCode │ MfgCode │ Language │ ApplicationType  │ OpSystemCode │ OpSystemName │ OpSystemVersion │ MfgCode │ MfgCode │  MfgName  │
│                 varchar                  │             varchar              │ varchar  │   varchar   │ varchar  │   varchar   │   varchar    │   varchar   │   varchar   │    varchar     │    varchar     │   varchar    │ varchar │ varchar  │     varchar      │   varchar    │   varchar    │     varchar     │ varcha